# MVP Pipeline de Dados
## Pesquisa sobre aparelhos celulares

Edmilson Prata da Silva

PUC-RJ - MBA em Ciência de Dados e Analytics

Disciplina de Engenharia de Dados

## Script para criação da base de dados SILVER

### Imports

Imports das bibliotecas necessárias para o funcionamento do script.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DecimalType

### Criando base de dados SILVER

Na camada silver os dados são persistidos após terem sido tratados, transformados e enriquecidos para o trabalho de analytics.

Nesta caso, a base original é a "Mobiles Dataset (2025)", do Kaggle. Ela foi persistida, mantendo suas características originais na camada bronze. Após, os dados sofrerão os tratamentos necessários para armazenamento nesta camada.

In [0]:
%sql CREATE DATABASE IF NOT EXISTS silver;

### Criando as tabelas

A seguir o script para criação das tabelas da camada silver.

In [0]:
print("Iniciando criação da camada silver...")

# Iniciar uma sessão Spark
spark = SparkSession.builder.appName("CreateSmartphonesTables").getOrCreate()

# Criar a tabela de dimensão "company"
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.company (
    company_id STRING COMMENT 'Chave primária (UUID)',
    company_name STRING COMMENT 'Nome da empresa fabricante'
)
USING DELTA
""")

print('Tabela silver.company criada com sucesso.')

# Criar a tabela de dimensão "model"
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.model (
    model_id STRING COMMENT 'Chave primária (UUID)',
    model_name STRING COMMENT 'Nome do modelo',
    mobile_weight DECIMAL(10, 2) COMMENT 'Peso em gramas',
    ram INTEGER COMMENT 'Quantidade de memória RAM em GB',
    qt_total_cameras INTEGER COMMENT 'Quantidade total de câmeras',
    front_camera DECIMAL(10, 2) COMMENT 'Resolução da câmera frontal principal em MP',
    back_camera DECIMAL(10, 2) COMMENT 'Resolução da câmera traseira principal em MP',
    processor STRING COMMENT 'Processador do aparelho',
    battery_capacity INTEGER COMMENT 'Capacidade da bateria em mAh',
    screen_size DECIMAL(4, 2) COMMENT 'Tamanho da tela',
    launched_year INTEGER COMMENT 'Ano de lançamento'
)
USING DELTA
""")

print('Tabela silver.model criada com sucesso.')

# Criar a tabela de dimensão "price"
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.price (
    price_id STRING COMMENT 'Chave primária (UUID)',
    model_id STRING COMMENT 'Chave estrangeira referenciando a tabela model',
    country STRING COMMENT 'País onde o preço foi registrado',
    launched_price DECIMAL(10, 2) COMMENT 'Preço de lançamento no país'
)
USING DELTA
""")

print('Tabela silver.price criada com sucesso.')

# Criar a tabela de fatos "smartphones"
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.smartphones (
    smartphone_id STRING COMMENT 'Chave primária (UUID)',
    company_id STRING COMMENT 'Chave estrangeira referenciando a tabela company',
    model_id STRING COMMENT 'Chave estrangeira referenciando a tabela model'
)
USING DELTA
""")

print('Tabela silver.smartphones criada com sucesso.')

# Exibir mensagem de sucesso
print("Tabelas criadas com sucesso!")

Iniciando criação da camada silver...
Tabela silver.company criada com sucesso.
Tabela silver.model criada com sucesso.
Tabela silver.price criada com sucesso.
Tabela silver.smartphones criada com sucesso.
Tabelas criadas com sucesso!


### Teste de Criação da Tabela
Teste de acesso a tabela recém criada, para garantir o sucesso da operação.

In [0]:
spark.sql("DESCRIBE TABLE silver.company").show()

+------------+---------+--------------------+
|    col_name|data_type|             comment|
+------------+---------+--------------------+
|  company_id|   string|Chave primária (U...|
|company_name|   string|Nome da empresa f...|
+------------+---------+--------------------+



In [0]:
spark.sql("DESCRIBE TABLE silver.model").show()

+----------------+-------------+--------------------+
|        col_name|    data_type|             comment|
+----------------+-------------+--------------------+
|        model_id|       string|Chave primária (U...|
|      model_name|       string|      Nome do modelo|
|   mobile_weight|decimal(10,2)|      Peso em gramas|
|             ram|          int|Quantidade de mem...|
|qt_total_cameras|          int|Quantidade total ...|
|    front_camera|decimal(10,2)|Resolução da câme...|
|     back_camera|decimal(10,2)|Resolução da câme...|
|       processor|       string|Processador do ap...|
|battery_capacity|          int|Capacidade da bat...|
|     screen_size| decimal(4,2)|     Tamanho da tela|
|   launched_year|          int|   Ano de lançamento|
+----------------+-------------+--------------------+



In [0]:
spark.sql("DESCRIBE TABLE silver.price").show()

+--------------+-------------+--------------------+
|      col_name|    data_type|             comment|
+--------------+-------------+--------------------+
|      price_id|       string|Chave primária (U...|
|      model_id|       string|Chave estrangeira...|
|       country|       string|País onde o preço...|
|launched_price|decimal(10,2)|Preço de lançamen...|
+--------------+-------------+--------------------+



In [0]:
spark.sql("DESCRIBE TABLE silver.smartphones").show()

+-------------+---------+--------------------+
|     col_name|data_type|             comment|
+-------------+---------+--------------------+
|smartphone_id|   string|Chave primária (U...|
|   company_id|   string|Chave estrangeira...|
|     model_id|   string|Chave estrangeira...|
+-------------+---------+--------------------+

